In [5]:
include("NYC.jl")

sales_dicts (generic function with 1 method)

In [6]:
borders_dict = read_borders()
;

# NYC

In [7]:
NYC_sales = read_processed_sales()
;

In [8]:
filtered = filter_sales(NYC_sales)
for_mm = filtered[:filtered]
;

In [9]:
model_dicts = sales_dicts(for_mm)
X_dict = model_dicts[:X_dict]
Y_dict = model_dicts[:Y_dict]
;

District 01 has 5 sales
District 02 has 85 sales
District 03 has 34 sales
District 04 has 6 sales
District 05 has 39 sales
District 06 has 30 sales
District 07 has 69 sales
District 08 has 739 sales
District 09 has 163 sales
District 10 has 312 sales
District 11 has 1172 sales
District 12 has 204 sales
District 13 has 322 sales
District 14 has 195 sales
District 15 has 517 sales
District 16 has 421 sales
District 17 has 459 sales
District 18 has 559 sales
District 19 has 686 sales
District 20 has 1010 sales
District 21 has 770 sales
District 22 has 1453 sales
District 23 has 245 sales
District 24 has 1283 sales
District 25 has 1317 sales
District 26 has 1268 sales
District 27 has 2199 sales
District 28 has 1033 sales
District 29 has 1956 sales
District 30 has 695 sales
District 31 has 0 sales
District 32 has 345 sales


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] anynull(::NullableArrays.NullableArray{Float64,1}) at ./deprecated.jl:57
 [3] convert(::Type{Array{Float64,2}}, ::DataTables.DataTable) at /Users/imolk/Library/Julia/packages/v0.6/DataTables/src/abstractdatatable/abstractdatatable.jl:545
 [4] setindex!(::Dict{String,Array{Float64,2}}, ::DataTables.DataTable, ::String) at ./dict.jl:420
 [5] sales_dicts(::DataTables.DataTable) at /Users/imolk/Documents/Harvard/Miratrix/rdd-coverage/Maxime/NYC/NYC.jl:192
 [6] include_string(::String, ::String) at ./loading.jl:515
 [7] include_string(::Module, ::String, ::String) at /Users/imolk/Library/Julia/packages/v0.6/Compat/src/Compat.jl:577
 [8] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/imolk/Library/Julia/packages/v0.6/IJulia/src/execute_request.jl:154
 [9] eventloop(::ZMQ.Socket) at /Users/imolk/Library/Julia/packages/v0.6/IJulia/src/eventloop.jl:8
 [10] (::IJulia.##14#17)() at ./task.jl:335
while loading In[9], in

In [10]:
points_distr_dict = Dict{String,Vector{LibGEOS.Point}}()
for distr in keys(X_dict)
    X = X_dict[distr]
    points = [LibGEOS.Point(X[i,:]) for i in 1:size(X,1)]
    points_distr_dict[distr] = points
end

In [23]:
LibGEOS.project

No documentation found.

`LibGEOS.project` is a `Function`.

```
# 2 methods for generic function "project":
project(line::LibGEOS.LineString, point::LibGEOS.Point) in LibGEOS at /Users/imolk/Library/Julia/packages/v0.6/LibGEOS/src/geos_operations.jl:30
project(g::Ptr{Ptr{Void}}, p::Ptr{Ptr{Void}}) in LibGEOS at /Users/imolk/Library/Julia/packages/v0.6/LibGEOS/src/geos_functions.jl:286
```


In [26]:
LibGEOS.project(borders_dict[(19,24)], points_distr_dict["19"][5])

10754.89799903885

In [29]:
p = points_distr_dict["19"][5]
border = borders_dict[(19,24)]
LibGEOS.nearestPoints(border, p)

2-element Array{LibGEOS.Point,1}:
 LibGEOS.Point(Ptr{Ptr{Void}} @0x00007fbf9b6c7190)
 LibGEOS.Point(Ptr{Ptr{Void}} @0x00007fbf9b6b8650)

In [47]:
p = points_distr_dict["19"][5]
border = borders_dict[(19,27)]
LibGEOS.nearestPoints(border, p)

2-element Array{LibGEOS.Point,1}:
 LibGEOS.Point(Ptr{Ptr{Void}} @0x00007fbf9d23d990)
 LibGEOS.Point(Ptr{Ptr{Void}} @0x00007fbf9a1dd400)

In [35]:
near=LibGEOS.nearestPoints(p, border)

2-element Array{LibGEOS.Point,1}:
 LibGEOS.Point(Ptr{Ptr{Void}} @0x00007fbf9b2079e0)
 LibGEOS.Point(Ptr{Ptr{Void}} @0x00007fbf9b2cd3d0)

In [42]:
GeoInterface.coordinates(near[2])

3-element Array{Float64,1}:
   1.01297e6
   1.87887e5
 NaN        

In [11]:
border = borders_dict[(19,27)]
distances_19 = [LibGEOS.distance(p, border) for p in points_distr_dict["19"]]
distances_27 = [LibGEOS.distance(p, border) for p in points_distr_dict["27"]]

2199-element Array{Float64,1}:
  6873.79 
  7380.36 
  6078.08 
 18277.2  
 17863.4  
 17581.2  
 16604.5  
 16394.1  
 17982.5  
 13329.9  
 13564.9  
 13694.1  
 13318.5  
     ⋮    
   758.551
  1602.21 
  2328.67 
  5467.29 
  5531.76 
   332.906
   123.798
   123.798
   345.143
   843.309
  2072.74 
  2285.45 

In [12]:
# residuals
resid_multi = JSON.parsefile("NYC_data/processed/resid_multi.json")

Dict{String,Any} with 32 entries:
  "24" => Any[5.82215, 5.27046, 6.08294, 5.83966, 5.81488, 5.7995, 5.48396, 5.4…
  "29" => Any[4.65263, 5.41024, 5.49985, 5.56356, 5.47666, 5.65043, 5.49956, 5.…
  "32" => Any[6.4072, 6.13528, 6.83249, 5.36659, 6.4321, 6.4974, 6.28408, 6.231…
  "07" => Any[6.2389, 4.6823, 5.10428, 4.63921, 6.07306, 4.77229, 4.89776, 4.40…
  "12" => Any[4.56657, 3.94776, 5.84151, 5.30726, 4.15171, 5.49041, 4.18879, 4.…
  "20" => Any[5.85571, 6.34274, 5.38755, 5.76039, 5.8591, 3.01936, 5.99472, 5.5…
  "06" => Any[6.32931, 6.395, 6.19472, 6.2479, 6.71488, 5.75223, 6.04542, 6.254…
  "25" => Any[5.49132, 5.96967, 6.33895, 6.22021, 5.79593, 5.98009, 5.92144, 5.…
  "03" => Any[6.30079, 6.60451, 6.86331, 6.11695, 6.57142, 6.33068, 6.64541, 6.…
  "22" => Any[5.296, 5.56968, 5.17892, 5.7061, 5.55155, 5.63543, 5.58394, 5.694…
  "11" => Any[4.90563, 3.61091, 5.06685, 5.0512, 4.91019, 5.1655, 5.78627, 5.25…
  "23" => Any[4.95637, 5.10909, 4.78241, 4.43588, 4.93451, 4.95898, 4.84532

In [13]:
projected_df = DataFrame(distance    = [distances_19 ; -distances_27], 
                         Y           = [Y_dict["19"] ; Y_dict["27"]], 
                         Y_residuals = [resid_multi["19"] ; resid_multi["27"]])

,distance,Y,Y_residuals
1,8990.268229658846,5.807474933213331,5.46181241665287
2,9000.462622457448,5.692418716005887,5.306997277350792
3,8981.091735672015,6.844953027433553,6.459531588778457
4,8478.190949756836,5.026383651064394,4.640962212409299
5,9586.550504455552,5.377649215830168,5.126841469243449
6,9567.60613643241,5.293652937256681,5.042845190669962
7,8366.19000202173,5.403677882205862,5.1528701356191435
8,7685.194197717486,5.103725717162268,4.758063200601807
9,8079.089979759623,5.857569584219502,5.346343667575157
10,8162.612994010347,4.862215288977982,4.516552772417521


In [14]:
DataFrames.writetable("NYC_data/processed/projected_19_27.csv", projected_df)

  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
